# Tutorial on online bin packing problem
## Please open in Colab !!!
Five steps:
1. Implement a sampler
2. Implement a evaluator
3. Prepare a template heuristic
4. Choose the LLM-EPS method and complete the config, and run

## Preparation: download the project file from GitHub. And update system path.

Download code from GitHub.

In [ ]:
!rm -rf py-alevo
!git clone https://github.com/RayZhhh/py-alevo.git

In [ ]:
from __future__ import annotations

import sys
sys.path.append('/content/py-alevo/')
sys.path.append('/content/py-alevo/example/online_bin_packing')

from typing import Any, List

import alevo
from alevo.tools.profiler import TensorboardProfiler
from _obp_evaluate import evaluate
from alevo.tools.llm.llm_api_https import HttpsApi
a = HttpsApi(host='', key='', model='', timeout=30)

## 1. Implement a sampler
The sampler defines the way to use local LLM or LLM API. You should create a new Sampler class by implementing `alevo.base.Sampler`.
- You should implement "draw_sample" function, to let the package know how to get a LLM's response by given a prompt.
- If you want more acceleration (such as batch inference, multi-threading sampling) you can also override "draw_samples" function.
- The following example shows a fake sampler, which returns a random function in the database.

In [ ]:
import pickle
import random


class FakeSampler(alevo.base.Sampler):
    """We select random functions from rand_function.pkl
    This sampler can help you debug your method even if you don't have an LLM API / deployed local LLM.
    """

    def __init__(self):
        super().__init__()
        try:
            with open('data/rand_function.pkl', 'rb') as f:
                self._functions = pickle.load(f) 
        except:
            with open('/content/py-alevo/example/online_bin_packing/data/rand_function.pkl', 'rb') as f:
                self._functions = pickle.load(f)

    def draw_samples(self, prompts: List[str | Any], *args, **kwargs) -> List[str]:
        return super().draw_samples(prompts)

    def draw_sample(self, prompt: str | Any, *args, **kwargs) -> str:
        """Generate an LLM response of the given prompt.
        """
        return random.choice(self._functions)

In [ ]:
# test: get a sample
sampler = FakeSampler()
sample = sampler.draw_sample('Fake sampler does not need a prompt.')
print(sample)

The following example shows a sampler that uses GPT-3.5-turbo API. If you want to use this sampler in this notebook, please complete following two variables.

In [ ]:
api_endpoint: str = ''  # the ip of your API provider, no "https://", such as "api.bltcy.top".
api_key: str = ''  # your API key which may start with "sk-......"

In [ ]:
import time
import http.client
import json


class MySampler(alevo.base.Sampler):
    def __init__(self):
        super().__init__()

    def draw_samples(self, prompts: List[str | Any], *args, **kwargs) -> List[str]:
        return super().draw_samples(prompts)

    def draw_sample(self, prompt: str | Any, *args, **kwargs) -> str:
        while True:
            try:
                conn = http.client.HTTPSConnection(f'{api_endpoint}', timeout=30)
                payload = json.dumps({
                    'max_tokens': 512,
                    'model': 'gpt-3.5-turbo',
                    'messages': [{'role': 'user', 'content': prompt}]
                })
                headers = {
                    'Authorization': f'Bearer {api_key}',
                    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
                    'Content-Type': 'application/json'
                }
                conn.request('POST', '/v1/chat/completions', payload, headers)
                res = conn.getresponse()
                data = res.read().decode('utf-8')
                data = json.loads(data)
                response = data['choices'][0]['message']['content']
                return response
            except Exception as e:
                print(e)
                time.sleep(2)
                continue

In [ ]:
# test 'MySampler' if you have an API key
if api_key != '' and api_endpoint != '':
    sampler = MySampler()
    response = sampler.draw_sample('Hello!')
    print(response)

## 2. Implement an evaluator
The evaluator defines how to evaluate the generated heuristic function. You should create a new Evaluator class by implementing `alevo.base.Evaluator`. You should override "evaluate_program" function to specify. Return None if the function is invalid.

The `alevo.base.Evaluator` class provide acceleration and safe evaluation methods. You can use them by simply setting respective arguments. The commonly used two argument are:
- `use_numba_accelerate`: If set to True, we will wrap the heuristic function with '@numba.jit(nopython=True)'. Please note that not all functions support numba.jit(), so use it appropriately.
- `timeout_second`: Terminate the evaluation after timeout seconds. If set to `None`, the evaluator will wait until the evaluation finish.

For more arguments, please refer to docstring in `algo.base.Evaluator`.

In [ ]:
class OBPEvaluator(alevo.base.Evaluator):
    """Evaluator for online bin packing problem."""

    def __init__(self):
        super().__init__(
            use_numba_accelerate=True,
            timeout_seconds=10
        )
        try:
            with open('data/weibull_train.pkl', 'rb') as f:
                self._bin_packing_or_train = pickle.load(f)['weibull_5k_train']
        except:
            with open('/content/py-alevo/example/online_bin_packing/data/weibull_train.pkl', 'rb') as f:
                self._bin_packing_or_train = pickle.load(f)['weibull_5k_train']

    def evaluate_program(self, program_str: str, callable_func: callable) -> Any | None:
        """Evaluate a given function. You can use compiled function (function_callable),
        as well as the original function strings for evaluation.
        Args:
            program_str: The function in string. You can ignore this argument when implementation.
            callable_func: The callable Python function of your sampled heuristic function code. 
            You can call the program using 'program_callable(args..., kwargs...)'
        Return:
            Returns the fitness value. Return None if you think the result is invalid.
        """
        # we call the _obp_evaluate.evaluate function to evaluate the callable code
        return evaluate(self._bin_packing_or_train, callable_func)

In [ ]:
evaluator = OBPEvaluator()
secure_evaluator = alevo.base.SecureEvaluator(evaluator=evaluator, debug_mode=True)

In [ ]:
# test the evaluator
test_program = '''
import numpy as np

def priority(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns priority with which we want to add item to each bin.
    Args:
        item: Size of item to be added to the bin.
        bins: Array of capacities for each bin.
    Return:
        Array of same size as bins with priority score of each bin.
    """
    return bins - item
'''

res = secure_evaluator.evaluate_program(test_program)
print(res)

In [ ]:
# we test an invalid program
test_program = '''
import numpy as np

def priority(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns priority with which we want to add item to each bin.
    Args:
        item: Size of item to be added to the bin.
        bins: Array of capacities for each bin.
    Return:
        Array of same size as bins with priority score of each bin.
    """
    while True:
        pass
'''

res = secure_evaluator.evaluate_program(test_program)
print(res)

## 3. Implement a template program

In [ ]:
template_program = '''
import numpy as np

def priority(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns priority with which we want to add item to each bin.
    Args:
        item: Size of item to be added to the bin.
        bins: Array of capacities for each bin.
    Return:
        Array of same size as bins with priority score of each bin.
    """
    penalty = np.arange(len(bins), 0, -1)
    scores = bins / (bins - item) - penalty
    max_capacity_bins = np.where(bins == bins.max())[0]
    for idx in max_capacity_bins:
        scores[idx] = -np.inf
    return scores
'''

## 4. Choose the LLM-EPS method and complete the config, and run
Our package support multiprocess running. However, the Colab backend has limited CPU support, so we set num_evlauators to 2.

Common args in Config:
- `num_samplers`: number of threads used in sampling.
- `num_evaluators`: number of processes used in evaluation (supports using multi-core CPUs).

In [ ]:
def run_randsample():
    from alevo.method.randsample import RandSample, Config
    profiler = TensorboardProfiler(log_dir='')
    config = Config(
        num_samplers=4,
        num_evaluators=2,
    )
    randsample = RandSample(template_program, sampler, evaluator, profiler, config=config, max_sample_nums=100)
    randsample.run()


def run_hillclimb():
    from alevo.method.hillclimb import HillClimb, Config
    profiler = TensorboardProfiler(log_dir='')
    config = Config(
        num_samplers=4,
        num_evaluators=2,
    )
    hillclimb = HillClimb(template_program, sampler, evaluator, profiler, config=config, max_sample_nums=100)
    hillclimb.run()


def run_funsearch():
    from alevo.method.funsearch import FunSearch, Config
    profiler = TensorboardProfiler(log_dir='')
    config = Config(
        num_samplers=2,
        num_evaluators=2,
        samples_per_prompt=2
    )
    funsearch = FunSearch(template_program, sampler, evaluator, profiler, config=config, max_sample_nums=100)
    funsearch.run()


def run_eoh():
    # Please note that you can simply pass the template program without function body, such as:
    # --------------------------------------------------------------------------------------------
    # import numpy as np
    # 
    # def priority(item: float, bins: np.ndarray) -> np.ndarray:
    #     """Returns priority with which we want to add item to each bin.
    #     Args:
    #         item: Size of item to be added to the bin.
    #         bins: Array of capacities for each bin.
    #     Return:
    #         Array of same size as bins with priority score of each bin.
    #     """
    #     pass
    # --------------------------------------------------------------------------------------------
    # This is because EoH can sample a valid code by sampling.
    from alevo.method.eoh import EoH, Config
    profiler = TensorboardProfiler(log_dir='')
    config = Config(
        num_samplers=2,
        num_evaluators=2,
        pop_size=20
    )
    # You can choose to add some task descriptions or not.
    task_description = """Please help me design a heuristic function for Online Bin Packing function. Given a item with its size S, the heuristic function finds the most suitable bin with remaining capacity C to pack it. We want to design a heuristic function that evaluate the priority of bins to which we want to assign the item to each bin."""
    eoh = EoH(
        task_description=task_description,
        template_program=template_program,
        config=config,
        max_generations=3,
        sampler=sampler,
        evaluator=evaluator,
        profiler=profiler
    )
    eoh.run()

In [ ]:
# It should be noted that the if __name__ == '__main__' is required.
# Because the inner code uses multiprocess evaluation.
if __name__ == '__main__':
    # you can also try other LLM-EPS methods.
    run_randsample()